### CREATE FACT TABLE

**Read Silver data**

In [0]:
df_silver = spark.sql(
    "select * from parquet.`abfss://silver@azureproject8940.dfs.core.windows.net/carsales`"
)

In [0]:
df_silver.display()

**Read all the DIM Tables**

In [0]:
df_dealer=spark.sql("select * from `car-sales-catalg`.gold.dim_dealer")
df_branch=spark.sql("select * from `car-sales-catalg`.gold.dim_branch")
df_date=spark.sql("select * from `car-sales-catalg`.gold.dim_date")
df_model=spark.sql("select * from `car-sales-catalg`.gold.dim_model")
                  

**Bringing Key to Fact tables**

In [0]:
df_fact=df_silver.join(df_branch,df_silver['Branch_ID']==df_branch['Branch_ID'],how='left')\
                 .join(df_dealer,df_silver['Dealer_ID']==df_dealer['Dealer_ID'],how='left')\
                 .join(df_model,df_silver['Model_ID']==df_model['Model_ID'],how='left')\
                 .join(df_date,df_silver['Date_ID']==df_date['Date_ID'],how='left')\
                 .select(df_silver['Revenue'],df_silver['Units_Sold'],df_silver['Revene_Per_Unit'],df_branch['Dim_Branch_Key'],df_dealer['Dim_Dealer_Key'],df_model['Dim_Model_Key'],df_date['Dim_Date_Key'])

In [0]:
display(df_fact)

#### Writing Fact Table

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("Factsalestbl"):
    deltatbl = DeltaTable.forName(spark, "`car-sales-catalg`.gold.Factsalestbl")
    deltatbl.alias("trg").merge(
        df_fact.alias("src"),
        "trg.Dim_Branch_Key=src.Dim_Branch_Key and trg.Dim_Dealer_Key=src.Dim_Dealer_Key and trg.Dim_Model_Key=src.Dim_Model_Key and trg.Dim_Date_Key=src.Dim_Date_Key"
    ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else:
    df_fact.write.format("delta").mode("overwrite").option(
        "path", "abfss://gold@azureproject8940.dfs.core.windows.net/Factsalesdata"
    ).saveAsTable("`car-sales-catalg`.gold.Factsalestbl")

In [0]:
%sql
select * from `car-sales-catalg`.gold.Factsalestbl